<a href="https://colab.research.google.com/github/pauloprsdesouza/recsys-twitter-social-capital/blob/dev-colab/RecSysTwitterSocialCapital.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

In [ ]:
pip install emoji
pip install python-twitter-v2
pip install transformers

# Import necessary libraries

In [ ]:
import re
import string
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import datetime
import emoji
import math
from pytwitter.models import User, Tweet, TweetEntities, TweetEntitiesUrl, TweetPublicMetrics, TweetEntitiesMention, TweetEntitiesHashtag, TweetEntitiesAnnotation
from pytwitter import Api
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification

## Nltk Dependencies
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

stemmer = SnowballStemmer('english')
stop_words = set(stopwords.words('english'))

## Bert Configuration
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Api Bearer Definition

In [ ]:
api = Api(bearer_token="AAAAAAAAAAAAAAAAAAAAAMhqlAEAAAAA4Pqzn354Z5nlkP5lKaW98vzlVlA%3D7GIA03xacVKdFYTFg7qmgvWTZThpa2FFd4SNPUqP7uPK7Xjue5")

# Step 1: Collect Data

In [ ]:
public_tweets = []

def collect_data():
    public_tweets = api.search_tweets(query="recommender systems lang:en has:hashtags -is:retweet has:media", 
                                      expansions=["referenced_tweets.id.author_id","in_reply_to_user_id","attachments.media_keys","author_id","entities.mentions.username"], 
                                      user_fields=["created_at","entities","id","location","name","pinned_tweet_id","profile_image_url","protected","public_metrics","url","username","verified"],
                                      tweet_fields=["attachments","author_id","context_annotations","created_at","entities","geo","in_reply_to_user_id","lang","public_metrics","reply_settings","source"], 
                                      max_results=50)

# Setp 2: Pre-process Data

In [ ]:
def pre_processing():
    # Convert to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r"http\S+", "", text)

    # Remove mentions and hashtags
    text = re.sub(r"@\S+", "", text)
    text = re.sub

    # Remove punctuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)

    # Tokenize
    tokens = word_tokenize(text)

    # Remove stop words
    tokens = [token for token in tokens if not token in stop_words]

    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for token in tokens:
        lemma = lemmatizer.lemmatize(token, wordnet.VERB)
        if lemma == token:
            lemma = lemmatizer.lemmatize(token, wordnet.NOUN)
        if lemma == token:
            lemma = lemmatizer.lemmatize(token, wordnet.ADJ)
        if lemma == token:
            lemma = lemmatizer.lemmatize(token, wordnet.ADV)
        lemmas.append

    # Disambiguate polysemous words
    tokens = disambiguate_polysemous_words(tokens)

    # Expand synonyms
    tokens = expand_synonyms(tokens)

    return tokens